In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statistics
from datetime import datetime
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

import seaborn as sns
import os
import statistics

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit

from dateutil.relativedelta import relativedelta 
from scipy.optimize import minimize              

import statsmodels.formula.api as smf          
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product      

from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing, Holt

from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams

from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
import statsmodels.api as sm

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters
from sklearn.preprocessing import RobustScaler

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [3]:
import xgboost as xgb
from xgboost import plot_importance, plot_tree

In [4]:
def create_dataset(X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [5]:
def create_dataset2(df, inv):
    X = df[['year','month','day','hour','day_of_week','quarter','day_of_year','week_of_year']]
    y = df[[inv]]
    return X, y

In [6]:
from math import sqrt

def rmse(y_true, y_pred):
    squared_error = 0
    for i in range(len(y_true)):
        squared_error += (y_true[i] - y_pred[i]) ** 2
    root_mean_squared_error = sqrt(squared_error / len(y_true))
    return root_mean_squared_error

In [7]:
def moving_avg(rounds):
    y_hat['avg'] = 0
    for i in range(len(Valid)):
        y_pred = 0
        for j in range(rounds):
            y_pred = y_pred + Train.footfall[len(Train) - (j + 1) * len(Valid) + i]
        y_hat['avg'][i] = y_pred / rounds

In [8]:
def test_stationary(timeseries):
    #Determine rolling statistics
    rolmean = pd.Series(timeseries).rolling(window = 28).mean()
    rolstd = pd.Series(timeseries).rolling(window = 28).std()
    
    #Plot rolling Statistics
    orig = plt.plot(timeseries, color = "blue", label = "Original")
    mean = plt.plot(rolmean, color = "red", label = "Rolling Mean")
    std = plt.plot(rolstd, color = "black", label = "Rolling Std")
    plt.legend(loc = "best")
    plt.title("Rolling Mean and Standard Deviation")
    plt.show(block = False)
    
    #Perform Dickey Fuller test
    print("Results of Dickey Fuller test: ")
    dftest = adfuller(timeseries, autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistics', 'p-value', '# Lag Used', 'Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    print(dfoutput)

In [9]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

### Exclude Locations with Too Much Missing Data:

In [10]:
ff = pd.read_csv('stackfootfall.csv')
ff.timestamp = pd.to_datetime(ff.timestamp, format = '%Y-%m-%d %H:%M:%S') 

In [11]:
ff = ff.loc[(ff.timestamp >= '2018-01') & (ff.timestamp <= '2019-08-18 23:00')]

ff_count = ff.groupby('location')['footfall'].count() / max(ff.groupby('location')['footfall'].count())
limit1 = ff_count.loc[ff_count >= 0.6].to_frame()
ff_test = ff.loc[(ff.timestamp >= '2019-07-29') & (ff.timestamp <= '2019-08-18 23:00')]
ff_test_count = ff_test.groupby('location')['footfall'].count() / max(ff_test.groupby('location')['footfall'].count())
limit2 = ff_test_count.loc[ff_test_count >= 0.8].to_frame()
limit = pd.merge(limit1, limit2, how = 'inner', on = 'location')

ff = pd.merge(ff, limit, how = 'inner', on = 'location')
ff

,Unnamed: 0,timestamp,location,device,footfall,year,month,day,hour,day_of_week,footfall_x,footfall_y
0,1344,2018-01-01 00:00:00,5.0,1780.0,494.0,2018,1,1,0,1,0.892256,1.0
1,1345,2018-01-01 01:00:00,5.0,1780.0,899.0,2018,1,1,1,1,0.892256,1.0
2,1346,2018-01-01 02:00:00,5.0,1780.0,770.0,2018,1,1,2,1,0.892256,1.0
3,1347,2018-01-01 03:00:00,5.0,1780.0,599.0,2018,1,1,3,1,0.892256,1.0
4,1348,2018-01-01 04:00:00,5.0,1780.0,331.0,2018,1,1,4,1,0.892256,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3818059,233635,2019-08-18 19:00:00,1168.0,827.0,42.0,2019,8,18,19,7,0.619529,1.0
3818060,233636,2019-08-18 20:00:00,1168.0,827.0,10.0,2019,8,18,20,7,0.619529,1.0
3818061,233637,2019-08-18 21:00:00,1168.0,827.0,13.0,2019,8,18,21,7,0.619529,1.0
3818062,233638,2019-08-18 22:00:00,1168.0,827.0,1.0,2019,8,18,22,7,0.619529,1.0


In [12]:
pred_accuracy = pd.DataFrame(columns = ('location', 'Naive', 'Moving_Average', 'Expoential_Smoothing', 
                                        'SARIMA', 'LSTM', 'XGBoost', 'best', 'sig'))

In [13]:
ff.location.nunique()

304

In [14]:
ff.location.unique()

array([   5.,    6.,    7.,   31.,   54.,   62.,   78.,   87.,   88.,
         89.,   91.,   92.,   97.,  108.,  111.,  112.,  114.,  115.,
        116.,  117.,  119.,  123.,  126.,  127.,  145.,  146.,  147.,
        152.,  155.,  165.,  185.,  186.,  222.,  233.,  237.,  266.,
        268.,  273.,  281.,  305.,  306.,  357.,  376.,  389.,  390.,
        409.,  414.,  419.,  421.,  425.,  429.,  445.,  447.,  448.,
        450.,  451.,  488.,  489.,  490.,  491.,  493.,  495.,  497.,
        498.,  500.,  501.,  502.,  503.,  504.,  509.,  512.,  515.,
        516.,  519.,  521.,  522.,  523.,  524.,  526.,  527.,  536.,
        541.,  542.,  544.,  546.,  547.,  551.,  554.,  556.,  560.,
        562.,  563.,  565.,  566.,  573.,  574.,  576.,  579.,  581.,
        584.,  586.,  587.,  588.,  589.,  592.,  593.,  597.,  598.,
        599.,  605.,  608.,  610.,  611.,  614.,  618.,  620.,  634.,
        639.,  642.,  649.,  657.,  660.,  705.,  710.,  711.,  712.,
        714.,  715.,

In [15]:
def IdentifyBestPred(y_test, y_hat):
    best_pred = 'naive'
    for i in ['avg','Holt_Winter','SARIMA','LSTM','XGBoost']:
        ret = dm_test(y_test, y_hat[best_pred], y_hat[i], h = 1, crit="MSE")
        if (ret.DM > 0):
            best_pred = i
    
    pred_sig = True
    col = ['naive','avg','Holt_Winter','SARIMA','LSTM','XGBoost']
    col.remove(best_pred)
    for i in col:
        if (dm_test(y_test, y_hat[best_pred], y_hat[i], h = 1, crit="MSE").p_value > 0.05):
            pred_sig = False
            
    return best_pred, pred_sig

In [16]:
for loc in ff.location.unique()[158:]:

    ff_loc = ff.loc[(ff.location <= loc) & (ff.location >= loc)][[
            'timestamp','footfall','year','month','day','hour','day_of_week']]    
    ff_loc.index = ff_loc.timestamp
    ff_loc = ff_loc.resample('H').mean()

    ff_loc['footfall'].replace(0, np.nan, inplace = True)
    ff_loc['footfall'] = ff_loc['footfall'].interpolate(method='linear')
    ff_loc = ff_loc.reset_index(level = ['timestamp'])
    ff_loc.timestamp = pd.to_datetime(ff_loc.timestamp, format = '%Y-%m-%d %H:%M:%S')

    temp = ff_loc[['timestamp', 'footfall']]
    temp.index = temp.timestamp
    temp['year'] = temp.timestamp.dt.year
    temp['month'] = temp.timestamp.dt.month
    temp['day'] = temp.timestamp.dt.day
    temp['hour'] = temp.timestamp.dt.hour
    temp['day_of_week'] = temp.timestamp.dt.dayofweek + 1
    temp['quarter'] = temp.timestamp.dt.quarter
    temp['day_of_year'] = temp.timestamp.dt.dayofyear
    temp['week_of_year'] = temp.timestamp.dt.weekofyear

    Train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')]
    Valid = temp.loc[(temp.timestamp >= '2019-07-29 00:00') & (temp.timestamp <= '2019-08-18 23:00')]
    
    if len(Valid) > 0:
        train = Train.resample('D').mean()
        valid = Valid.resample('D').mean() 

        train['footfall'] = train['footfall'].interpolate(method='linear')
        valid['footfall'] = valid['footfall'].interpolate(method='linear')

        # Monday to Thursday Ratio
        mtt = Train.loc[(Train.day_of_week >= 1) & (Train.day_of_week <= 4)].groupby('hour')['footfall'].mean().to_frame()
        mtt['ratio'] = mtt['footfall']/mtt['footfall'].sum()
        mtt = mtt.reset_index(level = ['hour'])

        # Friday Ratio
        fri = Train.loc[(Train.day_of_week >= 5) & (Train.day_of_week <= 5)].groupby('hour')['footfall'].mean().to_frame()
        fri['ratio'] = fri['footfall']/fri['footfall'].sum()
        fri = fri.reset_index(level = ['hour'])

        # Sat Ratio
        sat = Train.loc[(Train.day_of_week >= 6) & (Train.day_of_week <= 6)].groupby('hour')['footfall'].mean().to_frame()
        sat['ratio'] = sat['footfall']/sat['footfall'].sum()
        sat = sat.reset_index(level = ['hour'])

        # Sun Ratio
        sun = Train.loc[(Train.day_of_week >= 7) & (Train.day_of_week <= 7)].groupby('hour')['footfall'].mean().to_frame()
        sun['ratio'] = sun['footfall']/sun['footfall'].sum()
        sun = sun.reset_index(level = ['hour'])

        ratio1 = mtt[['hour','ratio']]
        ratio1['day_of_week'] = 1
        ratio2 = mtt[['hour','ratio']]
        ratio2['day_of_week'] = 2
        ratio3 = mtt[['hour','ratio']]
        ratio3['day_of_week'] = 3
        ratio4 = mtt[['hour','ratio']]
        ratio4['day_of_week'] = 4
        ratio5 = fri[['hour','ratio']] 
        ratio5['day_of_week'] = 5
        ratio6 = sat[['hour','ratio']]
        ratio6['day_of_week'] = 6
        ratio7 = sun[['hour','ratio']]
        ratio7['day_of_week'] = 7

        ratio_week = ratio1.append(ratio2).append(ratio3).append(ratio4).append(ratio5).append(ratio6).append(ratio7)

        merge = pd.merge(Valid, ratio_week, on = ('hour','day_of_week'), how = 'left')

        # Naive Approach
        dd = np.asarray(Train['footfall'])
        y_hat = Valid.copy()
        y_hat['naive']= dd[len(dd)- 1]
        for i in range(len(Valid)): 
            y_hat['naive'][i]= dd[len(dd) - len(Valid) + i]
        naive_rmse = rmse(Valid.footfall, y_hat.naive)


        # Moving Average Approach    
        moving_avg(4)
        avg4_rmse = rmse(Valid.footfall, y_hat.avg)
        moving_avg(6)
        avg6_rmse = rmse(Valid.footfall, y_hat.avg)    
        moving_avg(8)
        avg8_rmse = rmse(Valid.footfall, y_hat.avg)
        moving_avg(10)
        avg10_rmse = rmse(Valid.footfall, y_hat.avg)

        avg_rmse = min(avg4_rmse, avg6_rmse, avg8_rmse, avg10_rmse) 
        ma_temp = [avg4_rmse, avg6_rmse, avg8_rmse, avg10_rmse] 
        windows = (ma_temp.index(avg_rmse) + 2) * 2   
        moving_avg(windows)


        # Exponential Smoothing
        y_hat2 = valid.copy()
        fit1 = ExponentialSmoothing(np.asarray(train.footfall), seasonal_periods = 7, trend = 'add', seasonal= 'add').fit()
        y_hat2['Holt_Winter'] = fit1.forecast(len(valid))

        pred_d = y_hat2.Holt_Winter.to_frame()
        pred_d = pred_d.reset_index(level = ['timestamp'])
        pred_d['month'] = pred_d.timestamp.dt.month
        pred_d['day'] = pred_d.timestamp.dt.day
        temp1 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
        temp1['prediction'] = temp1['ratio'] * temp1['Holt_Winter'] * 24
        temp1.index = temp1.timestamp_x
        y_hat['Holt_Winter'] = temp1['prediction']
        holt_winter_rmse = rmse(Valid.footfall, y_hat['Holt_Winter'])

        # SARIMA
        arima_model = auto_arima(train.footfall, X = None, start_p = 0, d = 1, 
                         start_q = 0, max_p = 5, max_d=2, max_q=5, start_P = 0, 
                         D = 1, start_Q = 0, max_P = 5, max_D = 5, max_Q = 5, 
                         m = 7, seasonal = True, stepwise = True, n_jobs = 1, 
                         error_action = 'warn', trace = True, random = False, 
                         random_state = 20, n_fits = 50 )
        
        y_hat2['SARIMA'] = arima_model.predict(n_periods = len(valid))
        
        pred_d = pd.DataFrame(y_hat2['SARIMA'], index = valid.index)
        pred_d = pred_d.reset_index(level = ['timestamp'])
        pred_d['month'] = pred_d.timestamp.dt.month
        pred_d['day'] = pred_d.timestamp.dt.day
        temp2 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
        temp2['prediction'] = temp2['ratio'] * temp2['SARIMA'] * 24
        temp2.index = temp2.timestamp_x
        y_hat['SARIMA'] = temp2['prediction']
        sarima_rmse = rmse(Valid.footfall, y_hat['SARIMA'])

        # LSTM 
        train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')][[
            'footfall','month','day','hour','day_of_week']]
        test = temp.loc[(temp.timestamp >= '2019-07-28 14:00') & (temp.timestamp <= '2019-08-18 23:00')][[
            'footfall','month','day','hour','day_of_week']]  

        ff_transformer = RobustScaler()
        ff_transformer = ff_transformer.fit(train[['footfall']])

        train['footfall'] = ff_transformer.transform(train[['footfall']])
        test['footfall'] = ff_transformer.transform(test[['footfall']])

        time_steps = 10
        X_train, y_train = create_dataset(train, train.footfall, time_steps)
        X_test, y_test = create_dataset(test, test.footfall, time_steps)

        model = keras.Sequential()
        model.add(
          keras.layers.Bidirectional(
            keras.layers.LSTM(
              units=128, 
              input_shape=(X_train.shape[1], X_train.shape[2])
            )
          )
        )
        model.add(keras.layers.Dropout(rate=0.2))
        model.add(keras.layers.Dense(units=1))
        model.compile(loss='mean_squared_error', optimizer='adam')    

        history = model.fit(
        X_train, y_train, 
        epochs = 30, 
        batch_size = 32, 
        validation_split = 0.1,
        shuffle = False
        )

        y_pred = model.predict(X_test)

        y_train_inv = ff_transformer.inverse_transform(y_train.reshape(1, -1))
        y_test_inv = ff_transformer.inverse_transform(y_test.reshape(1, -1))
        y_pred_inv = ff_transformer.inverse_transform(y_pred)

        y_hat['LSTM'] =  y_pred_inv.flatten()
        lstm_rmse = rmse(y_test_inv.flatten(), y_pred_inv.flatten())
        
        # XGBoost
        train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')][[
                'footfall','year','month','day','hour','day_of_week','quarter','day_of_year','week_of_year']]
        test = temp.loc[(temp.timestamp >= '2019-07-29 00:00') & (temp.timestamp <= '2019-08-18 23:00')][[
                'footfall','year','month','day','hour','day_of_week','quarter','day_of_year','week_of_year']]  

        X_train, y_train = create_dataset2(train, 'footfall')
        X_test, y_test = create_dataset2(test, 'footfall')
        
        reg = xgb.XGBRegressor(n_estimators=1000)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                early_stopping_rounds=50, verbose=False)
        
        y_hat['XGBoost'] = reg.predict(X_test)
        xgboost_rmse = rmse(Valid.footfall, y_hat['XGBoost'])
        
        
        # save prediction results
        pred_loc = y_hat[['naive','avg','Holt_Winter','SARIMA','LSTM','XGBoost']]
        pred_loc.to_csv( 'pred3/'+ str(loc) + '.csv')
    
        # identify best prediction method
        best_pred, pred_sig = IdentifyBestPred(Valid.footfall, y_hat)
        pred_accuracy = pred_accuracy.append({'location': loc, 'Naive': naive_rmse, 'Moving_Average': avg_rmse, 
                                              'Expoential_Smoothing': holt_winter_rmse, 'SARIMA': sarima_rmse, 
                                              'LSTM': lstm_rmse, 'XGBoost':xgboost_rmse, 'best':best_pred, 
                                              'sig':pred_sig}, ignore_index=True)
        
        pred_accuracy = pred_accuracy.drop_duplicates(subset='location')
        pred_accuracy.to_csv('pred2/pred_accuracy.csv')

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=5295.779, Time=0.31 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=5057.005, Time=0.22 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=0.33 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=5217.862, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=5027.221, Time=0.24 sec
 ARIMA(1,1,0)(3,1,0)[7]             : AIC=4980.961, Time=0.47 sec
 ARIMA(1,1,0)(4,1,0)[7]             : AIC=4963.943, Time=0.77 sec
 ARIMA(1,1,0)(5,1,0)[7]             : AIC=4956.090, Time=1.16 sec
 ARIMA(1,1,0)(5,1,1)[7]             : AIC=inf, Time=4.82 sec
 ARIMA(1,1,0)(4,1,1)[7]             : AIC=inf, Time=3.14 sec
 ARIMA(0,1,0)(5,1,0)[7]             : AIC=5010.253, Time=0.95 sec
 ARIMA(2,1,0)(5,1,0)[7]             : AIC=4946.476, Time=1.61 sec
 ARIMA(2,1,0)(4,1,0)[7]             : AIC=4953.227, Time=0.85 sec
 ARIMA(2,1,0)(5,1,1)[7]             : AIC=inf, Time=6.10 sec
 ARIMA(2,1,0)(4,1,1)[7]             : AIC=inf, Time=3

 ARIMA(2,1,1)(5,1,1)[7]             : AIC=inf, Time=9.72 sec
 ARIMA(2,1,1)(4,1,1)[7]             : AIC=inf, Time=5.82 sec
 ARIMA(1,1,1)(5,1,0)[7]             : AIC=5117.757, Time=1.67 sec
 ARIMA(1,1,1)(4,1,0)[7]             : AIC=5129.848, Time=1.07 sec
 ARIMA(1,1,1)(5,1,1)[7]             : AIC=inf, Time=7.05 sec
 ARIMA(1,1,1)(4,1,1)[7]             : AIC=inf, Time=2.75 sec
 ARIMA(0,1,1)(5,1,0)[7]             : AIC=5121.862, Time=1.08 sec
 ARIMA(1,1,2)(5,1,0)[7]             : AIC=5119.407, Time=2.88 sec
 ARIMA(0,1,2)(5,1,0)[7]             : AIC=5117.412, Time=1.36 sec
 ARIMA(0,1,2)(4,1,0)[7]             : AIC=5129.366, Time=0.76 sec
 ARIMA(0,1,2)(5,1,1)[7]             : AIC=inf, Time=5.67 sec
 ARIMA(0,1,2)(4,1,1)[7]             : AIC=inf, Time=3.40 sec
 ARIMA(0,1,3)(5,1,0)[7]             : AIC=5119.408, Time=1.52 sec
 ARIMA(1,1,3)(5,1,0)[7]             : AIC=5121.398, Time=1.86 sec
 ARIMA(0,1,2)(5,1,0)[7] intercept   : AIC=5119.384, Time=4.44 sec

Best model:  ARIMA(0,1,2)(5,1,0)[7]    

388/388 [==============================] - 3s 9ms/step - loss: 0.0410 - val_loss: 0.0382
Epoch 16/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0402 - val_loss: 0.0375
Epoch 17/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0394 - val_loss: 0.0378
Epoch 18/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0387 - val_loss: 0.0426
Epoch 19/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0382 - val_loss: 0.0370
Epoch 20/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0386 - val_loss: 0.0366
Epoch 21/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0371 - val_loss: 0.0467
Epoch 22/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0359 - val_loss: 0.0451
Epoch 23/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0357 - val_loss: 0.0412
Epoch 24/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0350 - val_loss: 0.0396
Ep

 ARIMA(0,1,3)(0,1,2)[7]             : AIC=6409.913, Time=1.13 sec
 ARIMA(1,1,1)(0,1,2)[7]             : AIC=inf, Time=1.99 sec
 ARIMA(1,1,3)(0,1,2)[7]             : AIC=6412.185, Time=1.61 sec
 ARIMA(0,1,2)(0,1,2)[7] intercept   : AIC=6410.161, Time=1.32 sec

Best model:  ARIMA(0,1,2)(0,1,2)[7]          
Total fit time: 26.621 seconds
Epoch 1/30
388/388 [==============================] - 4s 10ms/step - loss: 0.3188 - val_loss: 0.1775
Epoch 2/30
388/388 [==============================] - 3s 9ms/step - loss: 0.1877 - val_loss: 0.1896
Epoch 3/30
388/388 [==============================] - 3s 9ms/step - loss: 0.1547 - val_loss: 0.1316
Epoch 4/30
388/388 [==============================] - 3s 9ms/step - loss: 0.1195 - val_loss: 0.0937
Epoch 5/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0944 - val_loss: 0.0630
Epoch 6/30
388/388 [==============================] - 3s 9ms/step - loss: 0.0796 - val_loss: 0.0621
Epoch 7/30
388/388 [==============================] - 3s 9ms/s

Epoch 13/30
387/387 [==============================] - 4s 9ms/step - loss: 0.1096 - val_loss: 0.1584
Epoch 14/30
387/387 [==============================] - 4s 10ms/step - loss: 0.1069 - val_loss: 0.1542
Epoch 15/30
387/387 [==============================] - 4s 9ms/step - loss: 0.1052 - val_loss: 0.1515
Epoch 16/30
387/387 [==============================] - 4s 9ms/step - loss: 0.1036 - val_loss: 0.1466
Epoch 17/30
387/387 [==============================] - 4s 10ms/step - loss: 0.1017 - val_loss: 0.1453
Epoch 18/30
387/387 [==============================] - 4s 9ms/step - loss: 0.1018 - val_loss: 0.1303
Epoch 19/30
387/387 [==============================] - 4s 9ms/step - loss: 0.1010 - val_loss: 0.1325
Epoch 20/30
387/387 [==============================] - 4s 9ms/step - loss: 0.0996 - val_loss: 0.1474
Epoch 21/30
387/387 [==============================] - 4s 9ms/step - loss: 0.0985 - val_loss: 0.1427
Epoch 22/30
387/387 [==============================] - 4s 9ms/step - loss: 0.0988 - val_l

...

 ARIMA(2,1,1)(1,1,2)[7]             : AIC=2671.726, Time=1.60 sec
 ARIMA(1,1,2)(1,1,2)[7]             : AIC=2671.023, Time=1.54 sec
 ARIMA(1,1,2)(0,1,2)[7]             : AIC=2673.550, Time=1.34 sec
 ARIMA(1,1,2)(1,1,1)[7]             : AIC=2673.510, Time=0.85 sec
 ARIMA(1,1,2)(2,1,2)[7]             : AIC=2672.840, Time=2.28 sec
 ARIMA(1,1,2)(1,1,3)[7]             : AIC=2672.876, Time=2.61 sec
 ARIMA(1,1,2)(0,1,1)[7]             : AIC=2673.988, Time=0.45 sec
 ARIMA(1,1,2)(0,1,3)[7]             : AIC=2675.547, Time=2.14 sec
 ARIMA(1,1,2)(2,1,1)[7]             : AIC=2675.473, Time=1.49 sec
 ARIMA(1,1,2)(2,1,3)[7]             : AIC=inf, Time=3.45 sec
 ARIMA(0,1,2)(1,1,2)[7]             : AIC=2673.977, Time=0.96 sec
 ARIMA(2,1,2)(1,1,2)[7]             : AIC=inf, Time=2.33 sec
 ARIMA(1,1,3)(1,1,2)[7]             : AIC=2677.514, Time=2.29 sec
 ARIMA(0,1,3)(1,1,2)[7]             : AIC=2674.155, Time=1.57 sec
 ARIMA(2,1,3)(1,1,2)[7]             : AIC=2674.874, Time=2.52 sec
 ARIMA(1,1,2)(1,1,2)

 ARIMA(1,1,1)(1,1,2)[7]             : AIC=3278.457, Time=1.10 sec
 ARIMA(1,1,1)(0,1,0)[7]             : AIC=inf, Time=0.20 sec
 ARIMA(1,1,1)(0,1,2)[7]             : AIC=3276.832, Time=0.80 sec
 ARIMA(1,1,1)(2,1,0)[7]             : AIC=3297.460, Time=0.96 sec
 ARIMA(1,1,1)(2,1,2)[7]             : AIC=3274.598, Time=1.30 sec
 ARIMA(1,1,1)(3,1,2)[7]             : AIC=3267.380, Time=2.75 sec
 ARIMA(1,1,1)(3,1,1)[7]             : AIC=inf, Time=2.65 sec
 ARIMA(1,1,1)(4,1,2)[7]             : AIC=3268.322, Time=5.70 sec
 ARIMA(1,1,1)(3,1,3)[7]             : AIC=3270.371, Time=4.40 sec
 ARIMA(1,1,1)(2,1,3)[7]             : AIC=3270.541, Time=4.29 sec
 ARIMA(1,1,1)(4,1,1)[7]             : AIC=3264.338, Time=5.09 sec
 ARIMA(1,1,1)(4,1,0)[7]             : AIC=3281.544, Time=2.77 sec
 ARIMA(1,1,1)(5,1,1)[7]             : AIC=inf, Time=6.25 sec
 ARIMA(1,1,1)(3,1,0)[7]             : AIC=3299.205, Time=1.50 sec
 ARIMA(1,1,1)(5,1,0)[7]             : AIC=3267.467, Time=4.25 sec
 ARIMA(1,1,1)(5,1,2)[7]  

 ARIMA(1,1,1)(5,1,1)[7]             : AIC=inf, Time=6.05 sec
 ARIMA(0,1,1)(4,1,0)[7]             : AIC=3235.839, Time=0.68 sec
 ARIMA(0,1,1)(3,1,0)[7]             : AIC=3256.214, Time=0.40 sec
 ARIMA(0,1,1)(5,1,0)[7]             : AIC=3236.814, Time=1.29 sec
 ARIMA(0,1,1)(4,1,1)[7]             : AIC=inf, Time=2.21 sec
 ARIMA(0,1,1)(3,1,1)[7]             : AIC=inf, Time=0.88 sec
 ARIMA(0,1,1)(5,1,1)[7]             : AIC=inf, Time=5.60 sec
 ARIMA(0,1,2)(4,1,0)[7]             : AIC=3236.127, Time=0.79 sec
 ARIMA(1,1,2)(4,1,0)[7]             : AIC=inf, Time=3.67 sec
 ARIMA(0,1,1)(4,1,0)[7] intercept   : AIC=3237.837, Time=1.66 sec

Best model:  ARIMA(0,1,1)(4,1,0)[7]          
Total fit time: 147.772 seconds
Epoch 1/30
250/250 [==============================] - 7s 29ms/step - loss: 0.2284 - val_loss: 0.2724
Epoch 2/30
250/250 [==============================] - 6s 25ms/step - loss: 0.0912 - val_loss: 0.2869
Epoch 3/30
250/250 [==============================] - 6s 25ms/step - loss: 0.0676 - 

240/240 [==============================] - 7s 29ms/step - loss: 0.3831 - val_loss: 0.1360
Epoch 2/30
240/240 [==============================] - 6s 26ms/step - loss: 0.1949 - val_loss: 0.1048
Epoch 3/30
240/240 [==============================] - 6s 27ms/step - loss: 0.1162 - val_loss: 0.0553
Epoch 4/30
240/240 [==============================] - 6s 26ms/step - loss: 0.0831 - val_loss: 0.0426
Epoch 5/30
240/240 [==============================] - 6s 27ms/step - loss: 0.0628 - val_loss: 0.0297
Epoch 6/30
240/240 [==============================] - 6s 27ms/step - loss: 0.0517 - val_loss: 0.0284
Epoch 7/30
240/240 [==============================] - 6s 26ms/step - loss: 0.0458 - val_loss: 0.0258
Epoch 8/30
240/240 [==============================] - 6s 26ms/step - loss: 0.0396 - val_loss: 0.0219
Epoch 9/30
240/240 [==============================] - 6s 26ms/step - loss: 0.0353 - val_loss: 0.0213
Epoch 10/30
240/240 [==============================] - 6s 26ms/step - loss: 0.0316 - val_loss: 0.0225


 ARIMA(1,1,0)(2,1,0)[7]             : AIC=3776.072, Time=0.20 sec
 ARIMA(1,1,0)(3,1,0)[7]             : AIC=3754.157, Time=0.31 sec
 ARIMA(1,1,0)(4,1,0)[7]             : AIC=3750.629, Time=0.61 sec
 ARIMA(1,1,0)(5,1,0)[7]             : AIC=3745.728, Time=1.01 sec
 ARIMA(1,1,0)(5,1,1)[7]             : AIC=inf, Time=2.84 sec
 ARIMA(1,1,0)(4,1,1)[7]             : AIC=inf, Time=2.00 sec
 ARIMA(0,1,0)(5,1,0)[7]             : AIC=3792.949, Time=0.85 sec
 ARIMA(2,1,0)(5,1,0)[7]             : AIC=3725.942, Time=1.22 sec
 ARIMA(2,1,0)(4,1,0)[7]             : AIC=3727.684, Time=0.79 sec
 ARIMA(2,1,0)(5,1,1)[7]             : AIC=inf, Time=3.56 sec
 ARIMA(2,1,0)(4,1,1)[7]             : AIC=inf, Time=2.10 sec
 ARIMA(3,1,0)(5,1,0)[7]             : AIC=3726.235, Time=1.30 sec
 ARIMA(2,1,1)(5,1,0)[7]             : AIC=inf, Time=2.79 sec
 ARIMA(1,1,1)(5,1,0)[7]             : AIC=inf, Time=2.05 sec
 ARIMA(3,1,1)(5,1,0)[7]             : AIC=inf, Time=3.54 sec
 ARIMA(2,1,0)(5,1,0)[7] intercept   : AIC=372

In [17]:
pred_accuracy = pred_accuracy.drop_duplicates(subset='location') 
pred_accuracy

,location,Naive,Moving_Average,Expoential_Smoothing,SARIMA,LSTM,XGBoost,best,sig
0,824.0,43.570027,43.949206,35.065502,37.033913,36.095560,37.048427,Holt_Winter,False
1,825.0,60.942128,49.005653,52.549287,50.995871,39.669983,45.537439,LSTM,True
2,829.0,66.042465,42.803260,37.354456,40.983987,31.442680,38.008978,LSTM,True
3,831.0,78.866562,65.745556,81.446080,83.256568,57.198923,60.806017,LSTM,False
4,832.0,83.098908,56.007812,56.303232,55.902625,52.851519,49.927019,XGBoost,True
...,...,...,...,...,...,...,...,...,...
141,1165.0,45.749615,43.096934,44.014018,42.906020,34.355755,43.018241,LSTM,False
142,1119.0,187.158854,155.810700,187.292793,185.555152,132.577826,141.014316,LSTM,False
143,1166.0,427.750929,376.716058,303.002030,305.690885,295.590579,314.020747,LSTM,False
144,1167.0,190.021774,162.629115,205.293522,201.795573,142.582352,158.112621,LSTM,False
